In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

print("Gerando DataFrame de simulação 'df_plot_data'...")

N_TRANSACOES = 100000
N_CONTAS = 20000
N_CLIENTES = 15000

# --- Simular dados de Contas e Chaves (para Gráficos 1 e 2) ---
df_contas = pd.DataFrame({
    'id_conta': [f'c_{i}' for i in range(N_CONTAS)],
    'is_high_risk': np.random.choice([0, 1], N_CONTAS, p=[0.97, 0.03])
})

# Simular Data de Abertura (para Gráfico 2)
# --- CORREÇÃO APLICADA AQUI ---
hoje = pd.to_datetime('2023-01-01')
# -----------------------------

dias_risco = pd.to_timedelta(np.random.randint(1, 181, N_CONTAS), unit='d')
dias_normal = pd.to_timedelta(np.random.randint(1, 365 * 10, N_CONTAS), unit='d')
df_contas['data_abertura'] = hoje - np.where(df_contas['is_high_risk'] == 1, dias_risco, dias_normal)

# Simular Delta de Cadastro de Chave (para Gráfico 1)
delta_dias_risco = pd.to_timedelta(np.random.randint(1, 6, N_CONTAS), unit='d')
delta_dias_normal = pd.to_timedelta(np.random.randint(1, 91, N_CONTAS), unit='d')
df_contas['delta_dias_chave'] = np.where(df_contas['is_high_risk'] == 1, delta_dias_risco.days, delta_dias_normal.days)

# --- Simular dados de Clientes e Contas (para Gráfico 3) ---
df_clientes = pd.DataFrame({
    'id_cliente': [f'cli_{i}' for i in range(N_CLIENTES)],
    'id_natureza': np.random.choice([1, 2], N_CLIENTES, p=[0.8, 0.2]) # 1=PF, 2=PJ
})
# Atribuir clientes às contas
df_contas['id_cliente'] = np.random.choice(df_clientes['id_cliente'], N_CONTAS)
# Juntar para obter a natureza
df_contas_clientes = df_contas.merge(df_clientes, on='id_cliente')
# Calcular o número de contas por cliente
df_contas_por_cliente = df_contas_clientes.groupby(['id_cliente', 'id_natureza']).size().reset_index(name='num_contas')

# --- Simular dados de Transações (para Gráficos 4, 5, 6, 7) ---
df_plot_data = pd.DataFrame({
    'id_transacao': [f't_{i}' for i in range(N_TRANSACOES)],
    'id_conta_destino': np.random.choice(df_contas['id_conta'], N_TRANSACOES)
})

# Juntar com dados das contas de destino
df_plot_data = df_plot_data.merge(df_contas, left_on='id_conta_destino', right_on='id_conta')

# Simular Categoria de Risco (para Gráfico 6)
# (Simplificação da lógica P(F))
categorias_risco = ["3. Risco Baixo (Base)", "2. Risco Médio (Chave Recente)", "1. Alto Risco (Conta)"]
probs_risco = [0.85, 0.12, 0.03] # Proporções de transações
df_plot_data['categoria_risco'] = np.random.choice(categorias_risco, N_TRANSACOES, p=probs_risco)

# Simular 'is_fraud' com base na categoria (para Gráfico 6)
probs_fraude = {
    "1. Alto Risco (Conta)": 0.60, # p_risco
    "2. Risco Médio (Chave Recente)": 0.40, # p_recente
    "3. Risco Baixo (Base)": 0.005 # p_base
}
df_plot_data['is_fraud'] = df_plot_data['categoria_risco'].map(
    lambda cat: np.random.choice([0, 1], p=[1-probs_fraude[cat], probs_fraude[cat]])
)

# Simular Valor (para Gráficos 4 e 5)
multiplicador = np.ones(N_TRANSACOES)
# Outliers Legítimos
outlier_legit_mask = (df_plot_data['is_fraud'] == 0) & (np.random.rand(N_TRANSACOES) < 0.04)
multiplicador = np.where(outlier_legit_mask, 2.5, multiplicador)
# Fraudes
fraude_mask = df_plot_data['is_fraud'] == 1
multiplicador = np.where(fraude_mask, 30.0, multiplicador)
# Criar coluna de tipo (para Gráfico 4)
df_plot_data['tipo_transacao_plot'] = np.where(fraude_mask, "Fraude", np.where(outlier_legit_mask, "Outlier Legítimo", "Legítima"))

# Valor base Log-Normal
valor_base = np.random.lognormal(mean=np.log(150), sigma=0.8, size=N_TRANSACOES)
df_plot_data['valor'] = valor_base * multiplicador

# Injetar fraudes "Abaixo do Radar" (para Gráfico 5)
abaixo_radar_mask = (df_plot_data['is_fraud'] == 1) & (np.random.rand(N_TRANSACOES) < 0.4)
valores_limite = [499.90, 999.90, 1999.90, 4999.90]
df_plot_data['valor'] = np.where(
    abaixo_radar_mask,
    np.random.choice(valores_limite, N_TRANSACOES),
    df_plot_data['valor']
)
df_plot_data['valor'] = df_plot_data['valor'].round(2)

# Simular dados do Fan-Out (para Gráfico 7)
df_plot_data['fraud_type'] = np.where(df_plot_data['is_fraud'], 'triangulacao_conta_laranja', None)
# Apenas para o gráfico, vamos criar uma coluna que representa a contagem de filhas
# Em um cenário real, você faria o groupby
n_fraudes_cadeia = len(df_plot_data[df_plot_data['fraud_type'] == 'triangulacao_conta_laranja'])
df_fan_out_counts = pd.DataFrame({
    'num_filhas_fan_out': np.random.randint(2, 6, size=n_fraudes_cadeia) # U{2,3,4,5}
})


print("DataFrame de simulação 'df_plot_data' criado com sucesso.")
print(df_plot_data[['data_abertura', 'delta_dias_chave', 'is_high_risk', 'categoria_risco', 'is_fraud', 'valor', 'tipo_transacao_plot']].head())

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Configurações de Plotagem ---
sns.set_theme(style="whitegrid")
# Criar uma figura grande para 8 subplots (4 linhas, 2 colunas)
fig, axes = plt.subplots(4, 2, figsize=(20, 28))
fig.suptitle('Dashboard de Validação dos Dados Sintéticos', fontsize=24, weight='bold', y=1.03)
# Paletas de cores
palette_risco = {0: "royalblue", 1: "crimson"}
palette_natureza = {1: "mediumseagreen", 2: "darkorange"}
palette_tipo_tx = {"Legítima": "cornflowerblue", "Outlier Legítimo": "orange", "Fraude": "red"}

# --- Gráfico 1: Delta Cadastro de Chave PIX (Risco vs. Normal) ---
ax1 = axes[0, 0]
sns.kdeplot(
    data=df_contas, 
    x='delta_dias_chave', 
    hue='is_high_risk',
    palette=palette_risco,
    fill=True, 
    common_norm=False,
    ax=ax1
)
ax1.set_title('Gráfico 1: Delta (Dias) para Cadastro de Chave PIX', fontsize=16)
ax1.set_xlabel('Dias entre Abertura da Conta e Cadastro da Chave', fontsize=12)
ax1.set_ylabel('Densidade', fontsize=12)
ax1.legend(title='Risco da Conta', labels=['Alto Risco (1)', 'Normal (0)'])

# --- Gráfico 2: Data de Abertura da Conta (Risco vs. Normal) ---
ax2 = axes[0, 1]
sns.histplot(
    data=df_contas,
    x='data_abertura',
    hue='is_high_risk',
    palette=palette_risco,
    multiple='stack',
    bins=100,
    ax=ax2
)
ax2.set_title('Gráfico 2: Distribuição da Data de Abertura de Conta', fontsize=16)
ax2.set_xlabel('Data de Abertura', fontsize=12)
ax2.set_ylabel('Contagem de Contas', fontsize=12)
ax2.legend(title='Risco da Conta', labels=['Alto Risco (1)', 'Normal (0)'])

# --- Gráfico 3: Contagem de Contas por Cliente (PF vs. PJ) ---
ax3 = axes[1, 0]
# Precisamos plotar dois gráficos de barras separados, um para PF e outro para PJ
# Usaremos o `df_contas_por_cliente`
sns.countplot(
    data=df_contas_por_cliente[df_contas_por_cliente['id_natureza'] == 1],
    x='num_contas',
    ax=ax3,
    color='mediumseagreen'
)
ax3.set_title('Gráfico 3a: Contagem de Contas por Cliente (PF)', fontsize=16)
ax3.set_xlabel('Número de Contas (PF)', fontsize=12)
ax3.set_ylabel('Contagem de Clientes', fontsize=12)

ax4 = axes[1, 1]
sns.countplot(
    data=df_contas_por_cliente[df_contas_por_cliente['id_natureza'] == 2],
    x='num_contas',
    ax=ax4,
    color='darkorange'
)
ax4.set_title('Gráfico 3b: Contagem de Contas por Cliente (PJ)', fontsize=16)
ax4.set_xlabel('Número de Contas (PJ)', fontsize=12)
ax4.set_ylabel('Contagem de Clientes', fontsize=12)

# --- Gráfico 4: Distribuição de Valores (Log) por Tipo de Transação ---
ax5 = axes[2, 0]
# Usar log10 para melhor interpretação da escala de valor
df_plot_data['log10_valor'] = np.log10(df_plot_data['valor'].clip(lower=0.01))
sns.kdeplot(
    data=df_plot_data,
    x='log10_valor',
    hue='tipo_transacao_plot',
    palette=palette_tipo_tx,
    fill=True,
    common_norm=False,
    ax=ax5
)
ax5.set_title('Gráfico 4: Distribuição de Valores de Transação (Escala Log)', fontsize=16)
ax5.set_xlabel('Log10(Valor da Transação em R$)', fontsize=12)
ax5.set_ylabel('Densidade', fontsize=12)

# --- Gráfico 5: Histograma de Valores de Fraude "Abaixo do Radar" ---
ax6 = axes[2, 1]
sns.histplot(
    data=df_plot_data[df_plot_data['is_fraud'] == 1],
    x='valor',
    bins=100,
    ax=ax6,
    color='crimson'
)
ax6.set_xlim(0, 5500) # Focar na área dos valores limite
ax6.set_title('Gráfico 5: Valores de Fraude (Foco em "Abaixo do Radar")', fontsize=16)
ax6.set_xlabel('Valor da Transação (R$)', fontsize=12)
ax6.set_ylabel('Contagem de Fraudes', fontsize=12)
ax6.text(1000, ax6.get_ylim()[1]*0.8, 'Picos esperados em\n499, 999, 1999, 4999', 
         style='italic', bbox={'facecolor': 'white', 'alpha': 0.8, 'pad': 5})

# --- Gráfico 6: Taxa de Fraude Realizada por Categoria de Risco ---
ax7 = axes[3, 0]
# Agrupar por categoria e calcular a taxa de fraude
df_taxa_fraude = df_plot_data.groupby('categoria_risco')['is_fraud'].mean().reset_index().sort_values(by='categoria_risco')
sns.barplot(
    data=df_taxa_fraude,
    x='categoria_risco',
    y='is_fraud',
    ax=ax7,
    palette=['red', 'orange', 'green']
)
# Adicionar rótulos de porcentagem
for p in ax7.patches:
    ax7.annotate(f'{p.get_height():.1%}', (p.get_x() + p.get_width() / 2., p.get_height()),
                 ha='center', va='center', fontsize=12, color='black', xytext=(0, 5),
                 textcoords='offset points')
ax7.set_title('Gráfico 6: Taxa de Fraude Real vs. Categoria de Risco', fontsize=16)
ax7.set_xlabel('Categoria de Risco (Simulada pela lógica P(F))', fontsize=12)
ax7.set_ylabel('Taxa Média de Fraude (mean(is_fraud))', fontsize=12)
ax7.set_ylim(0, 1.0) # Eixo de 0% a 100%

# --- Gráfico 7: Histograma do "Fan-Out" (Triangulação) ---
ax8 = axes[3, 1]
# Usando o df_fan_out_counts simulado
sns.countplot(
    data=df_fan_out_counts,
    x='num_filhas_fan_out',
    ax=ax8,
    color='purple'
)
ax8.set_title('Gráfico 7: Distribuição do "Fan-Out" em Fraudes em Cadeia', fontsize=16)
ax8.set_xlabel('Número de Transações Filhas (k)', fontsize=12)
ax8.set_ylabel('Contagem de Fraudes Raiz', fontsize=12)

# --- Finalizar ---
plt.tight_layout(pad=3.0)
plt.show()